In [1]:
import time  # 导入时间模块

import numpy as np  # 导入NumPy库
from IPython.display import clear_output  # 导入clear_output用于清除输出
from pydrake.all import (
    AddMultibodyPlantSceneGraph,  # 添加多体植物和场景图
    DiagramBuilder,  # 图构建器
    JacobianWrtVariable,  # 雅可比变量类型
    MathematicalProgram,  # 数学规划
    MeshcatVisualizer,  # Meshcat可视化工具
    PiecewisePolynomial,  # 分段多项式
    Solve,  # 求解器
    StartMeshcat,  # 启动Meshcat
)

from manipulation import running_as_notebook  # 判断是否在notebook中运行
from manipulation.meshcat_utils import plot_mathematical_program  # 绘制数学规划
from manipulation.scenarios import AddTwoLinkIiwa  # 添加双连杆Iiwa机器人


# 这个函数是本notebook特有的，但我把它放在头部以减少干扰。
def Visualizer(MakeMathematicalProgram):
    builder = DiagramBuilder()  # 创建图构建器

    plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.0)  # 添加多体植物和场景图
    twolink = AddTwoLinkIiwa(plant, q0=[0.0, 0.0])  # 添加双连杆Iiwa，初始角度为[0, 0]
    hand = plant.GetFrameByName("iiwa_link_ee")  # 获取末端执行器的frame
    plant.Finalize()  # 完成plant的构建

    MeshcatVisualizer.AddToBuilder(builder, scene_graph, meshcat)  # 添加Meshcat可视化
    diagram = builder.Build()  # 构建图
    context = diagram.CreateDefaultContext()  # 创建默认上下文
    plant_context = plant.GetMyContextFromRoot(context)  # 获取plant的上下文

    meshcat.Delete()  # 清空Meshcat
    meshcat.SetProperty("/Background", "top_color", [0, 0, 0])  # 设置背景顶部颜色为黑色
    meshcat.SetProperty("/Background", "bottom_color", [0, 0, 0])  # 设置背景底部颜色为黑色
    meshcat.SetProperty("/Grid", "visible", False)  # 隐藏网格

    X, Y = np.meshgrid(np.linspace(-5, 5, 35), np.linspace(-5, 5, 31))  # 创建网格

    def visualize(q, v_Gdesired=[1.0, 0.0], t=None):
        if t:
            context.SetTime(t)  # 设置仿真时间
        plant.SetPositions(plant_context, q)  # 设置关节位置
        diagram.ForcedPublish(context)  # 强制刷新可视化

        J_G = plant.CalcJacobianTranslationalVelocity(
            plant_context,
            JacobianWrtVariable.kQDot,
            hand,
            [0, 0, 0],
            plant.world_frame(),
            plant.world_frame(),
        )
        J_G = J_G[[0, 2], :]  # 忽略Y方向，只取X和Z
        print("J_G = ")
        print(np.array2string(J_G, formatter={"float": lambda x: "{:5.2f}".format(x)}))

        prog = MakeMathematicalProgram(q, J_G, v_Gdesired)  # 构建数学规划问题
        result = Solve(prog)  # 求解数学规划
        plot_mathematical_program(meshcat, "QP", prog, X, Y, result=result)  # 可视化QP解
        # TODO: 添加set_object到meshcat.Animation
        if False:  # meshcat._is_recording:
            with meshcat._animation.at_frame(v, meshcat._recording_frame_num) as m:
                plot_mathematical_program(m, prog, X, Y, result=result)
        clear_output(wait=True)  # 清除输出，便于动态刷新

    return visualize  # 返回可视化函数

In [2]:
# 启动可视化工具。
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7003


# 将微分逆运动学表示为二次规划（QP）
 
## 在此处定义你的数学规划问题。


In [3]:
def MakeMathematicalProgram(q, J_G, v_Gdesired):
    prog = MathematicalProgram()  # 创建数学规划对象
    v = prog.NewContinuousVariables(2, "v")  # 新建2个连续变量v
    v_max = 3.0  # 最大速度约束

    error = J_G @ v - np.asarray(v_Gdesired)  # 误差项
    prog.AddCost(error.dot(error))  # 添加二次型代价
    prog.AddBoundingBoxConstraint(-v_max, v_max, v)  # 添加速度上下界约束

    return prog  # 返回数学规划对象

## 可视化某一组关节角

In [4]:
visualize = Visualizer(MakeMathematicalProgram)  # 获取可视化函数

q = [-np.pi / 2.0 + 0.5, 1.0]  # 关节角
v_Gdesired = [0.5, 0.0]  # 期望末端速度
visualize(q, v_Gdesired)  # 可视化

J_G = 
[[-0.05  0.25]
 [ 0.83 -0.46]]


## 动画关节轨迹（穿越奇异点）

In [6]:
visualize = Visualizer(MakeMathematicalProgram)  # 获取可视化函数

v_Gdesired = [1.0, 0.0]  # 期望末端速度
T = 2.0  # 时间长度
q = PiecewisePolynomial.FirstOrderHold(
    [0, T, 2 * T],
    np.array([
        [-np.pi / 2.0 + 1.0, -np.pi / 2.0 - 1.0, -np.pi / 2.0 + 1.0],
        [2.0, -2.0, 2],
    ]),
)

nx = 35
ny = 31
X, Y = np.meshgrid(np.linspace(-5, 5, nx), np.linspace(-5, 5, ny))
D = np.vstack((X.reshape(1, -1), Y.reshape(1, -1)))
for i in range(2):
    for t in np.linspace(0, 2 * T, num=100):
        visualize(q.value(t), v_Gdesired, t=t)  # 动画显示
        if not running_as_notebook:
            break
        time.sleep(0.05)

J_G = 
[[-0.09  0.44]
 [ 0.51 -0.28]]


## 轨迹滑块
 
TODO(russt)：一旦我能在meshcat动画中保存绘制的表面，就可以移除此部分。

In [19]:
visualize = Visualizer(MakeMathematicalProgram)  # 获取可视化函数

v_Gdesired = [1.0, 0.0]  # 期望末端速度
T = 2.0  # 时间长度
qtraj = PiecewisePolynomial.FirstOrderHold(
    [0, T], np.array([[-np.pi / 2.0 + 1.0, -np.pi / 2.0 - 1.0], [2.0, -2.0]])
)
visualize(qtraj.value(0), v_Gdesired)  # 初始可视化

meshcat.AddSlider("time", min=0, max=T, step=0.05, value=0)  # 添加时间滑块
meshcat.AddButton("Stop Interaction Loop")  # 添加停止按钮
while meshcat.GetButtonClicks("Stop Interaction Loop") < 1:
    t = meshcat.GetSliderValue("time")  # 获取当前滑块值
    visualize(qtraj.value(t), v_Gdesired)  # 动态可视化
    if not running_as_notebook:
        break
    time.sleep(0.05)
meshcat.DeleteAddedControls()  # 删除控件

J_G = 
[[-0.09  0.44]
 [ 0.51 -0.28]]


## 关节滑块

In [20]:
visualize = Visualizer(MakeMathematicalProgram)  # 获取可视化函数

q = [-np.pi / 2.0 + 0.5, 1.0]  # 初始关节角
v_Gdesired = [1.0, 0.0]  # 期望末端速度
visualize(q, v_Gdesired)  # 初始可视化


def _q_callback(change, index):
    q[index] = change.new
    visualize(q, v_Gdesired)


def _vG_callback(change, index):
    v_Gdesired[index] = change.new
    visualize(q, v_Gdesired)


meshcat.AddSlider("q0", value=q[0], min=-np.pi, max=np.pi, step=0.1)  # 添加q0滑块
meshcat.AddSlider("q1", value=q[1], min=-np.pi, max=np.pi, step=0.1)  # 添加q1滑块
meshcat.AddSlider("v_G_W0", value=v_Gdesired[0], min=-4, max=4, step=0.1)  # 添加末端速度X滑块
meshcat.AddSlider("v_G_W1", value=v_Gdesired[1], min=-4, max=4, step=0.1)  # 添加末端速度Z滑块

meshcat.AddButton("Stop Interaction Loop")  # 添加停止按钮
while meshcat.GetButtonClicks("Stop Interaction Loop") < 1:
    q = [meshcat.GetSliderValue("q0"), meshcat.GetSliderValue("q1")]  # 获取当前关节角
    v_Gdesired = [
        meshcat.GetSliderValue("v_G_W0"),
        meshcat.GetSliderValue("v_G_W1"),
    ]  # 获取当前末端速度
    visualize(q, v_Gdesired)  # 动态可视化
    if not running_as_notebook:
        break
    time.sleep(0.05)
meshcat.DeleteAddedControls()  # 删除控件

KeyboardInterrupt: 